<a href="https://colab.research.google.com/github/stevenbowler/LamdaLabsTest/blob/master/colabs/llama_2_7b_FineTuneDatasetPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb)

# Fine Tune LLaMa 7B model with RAG

SB: 09/1/2023 updated to use 7b LLaMa model, autotrain a model on the bigbio/med_qa dataset
---

🚨 _Note that running this on CPU is sloooow.

---

In [2]:
!pip install -qU \
  huggingface_hub \
  datasets \
  pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00


In [3]:
# connect to huggingface
from huggingface_hub import notebook_login
notebook_login()

Pull the dataset from HuggingFace

In [4]:
from datasets import load_dataset
dataset = load_dataset("bigbio/med_qa")

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/27a6d96c9db3334f9b3c1f99484adac7b7579fe06926d41e9005000368623ad4/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/27a6d96c9db3334f9b3c1f99484adac7b7579fe06926d41e9005000368623ad4/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/27a6d96c9db3334f9b3c1f99484adac7b7579fe06926d41e9005000368623ad4/data_clean/questions/US/dev.jsonl


In [48]:
# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

In [49]:
df.head()

,meta_info,question,answer_idx,answer,options
0,step2&3,A 23-year-old pregnant woman at 22 weeks gesta...,E,Nitrofurantoin,"[{'key': 'A', 'value': 'Ampicillin'}, {'key': ..."
1,step2&3,A 3-month-old baby died suddenly at night whil...,A,Placing the infant in a supine position on a f...,"[{'key': 'A', 'value': 'Placing the infant in ..."
2,step1,A mother brings her 3-week-old infant to the p...,A,Abnormal migration of ventral pancreatic bud,"[{'key': 'A', 'value': 'Abnormal migration of ..."
3,step1,A pulmonary autopsy specimen from a 58-year-ol...,A,Thromboembolism,"[{'key': 'A', 'value': 'Thromboembolism'}, {'k..."
4,step1,A 20-year-old woman presents with menorrhagia ...,E,Von Willebrand disease,"[{'key': 'A', 'value': 'Factor V Leiden'}, {'k..."


In [50]:
df['question'] = 'Question: ' + df['question']

In [51]:
df['question']

0        Question: A 23-year-old pregnant woman at 22 w...
1        Question: A 3-month-old baby died suddenly at ...
2        Question: A mother brings her 3-week-old infan...
3        Question: A pulmonary autopsy specimen from a ...
4        Question: A 20-year-old woman presents with me...
                               ...                        
10173    Question: A 60-year-old man presents to the em...
10174    Question: A 45-year-old male with a 15-year hi...
10175    Question: After receiving a positive newborn s...
10176    Question: A 25-year-old man comes to the offic...
10177    Question: A 26-year-old primigravid woman come...
Name: question, Length: 10178, dtype: object

In [52]:
df['answer'] = ' Answer: ' + df['answer']

In [53]:
df['text'] = df['question'] + df['answer']

In [55]:
df.drop(['question', 'answer', 'options'], axis=1, inplace=True)

In [56]:
df

,meta_info,answer_idx,text
0,step2&3,E,Question: A 23-year-old pregnant woman at 22 w...
1,step2&3,A,Question: A 3-month-old baby died suddenly at ...
2,step1,A,Question: A mother brings her 3-week-old infan...
3,step1,A,Question: A pulmonary autopsy specimen from a ...
4,step1,E,Question: A 20-year-old woman presents with me...
...,...,...,...
10173,step2&3,B,Question: A 60-year-old man presents to the em...
10174,step2&3,B,Question: A 45-year-old male with a 15-year hi...
10175,step1,B,Question: After receiving a positive newborn s...
10176,step1,C,Question: A 25-year-old man comes to the offic...


In [57]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
# Save the dataframe to a CSV file.
df.to_csv('/content/drive/MyDrive/ColabStuff/train.csv')

# then upload the file to HuggingFace DataSet